In [1]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor(torch.tensor(obs).unsqueeze(0))
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        env.render()
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()
    env.close()

c:\users\an_asus\appdata\local\programs\python\python39\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(
C:\Users\An_asus\AppData\Local\Temp/ipykernel_8372/4112361909.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  train_obs_v = torch.FloatTensor(train_obs)


0: loss=0.690, reward_mean=18.1, reward_bound=19.0
1: loss=0.675, reward_mean=25.8, reward_bound=29.5
2: loss=0.658, reward_mean=33.9, reward_bound=32.0
3: loss=0.653, reward_mean=36.5, reward_bound=36.0
4: loss=0.632, reward_mean=51.7, reward_bound=63.5
5: loss=0.625, reward_mean=45.9, reward_bound=49.5
6: loss=0.630, reward_mean=54.3, reward_bound=61.5
7: loss=0.608, reward_mean=53.1, reward_bound=60.5
8: loss=0.592, reward_mean=49.5, reward_bound=58.0
9: loss=0.586, reward_mean=58.4, reward_bound=62.0
10: loss=0.596, reward_mean=57.5, reward_bound=61.0
11: loss=0.582, reward_mean=53.8, reward_bound=71.5
12: loss=0.599, reward_mean=54.8, reward_bound=60.5
13: loss=0.569, reward_mean=83.7, reward_bound=102.5
14: loss=0.570, reward_mean=63.2, reward_bound=58.5
15: loss=0.576, reward_mean=92.8, reward_bound=113.5
16: loss=0.570, reward_mean=63.7, reward_bound=77.0
17: loss=0.551, reward_mean=70.4, reward_bound=60.5
18: loss=0.576, reward_mean=86.0, reward_bound=100.0
19: loss=0.553, rew

In [12]:
import numpy as np

In [10]:
env = gym.make('CartPole-v0')
obs = env.reset()
sm = nn.Softmax(dim = 1)

obs = torch.FloatTensor(torch.tensor(obs).unsqueeze(0))
np.argmax(sm(net(obs)).detach().numpy())

0

In [14]:
env = gym.make('CartPole-v0')
obs = env.reset()
sm = nn.Softmax(dim = 1)

while True:
    env.render()
    obs = torch.FloatTensor(torch.tensor(obs).unsqueeze(0))
    action = np.argmax(sm(net(obs)).detach().numpy())
    
    obs, reward, is_done, _ = env.step(action)
    if is_done:
        break
env.close()